In [1]:
import pandas as pd
import numpy as np

df_reviews = pd.read_csv("df_review_export_python.csv")

In [4]:
#!pip install transformers
from transformers import pipeline

#using model: https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english
pipe = pipeline("text-classification", model = "distilbert-base-uncased-finetuned-sst-2-english")


[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 8.2/8.2 MB 10.1 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 8.2 MB/s eta 0:00:00
     -------------------------------------- 277.3/277.3 kB 8.6 MB/s eta 0:00:00
     -------------------------------------- 311.7/311.7 kB 6.4 MB/s eta 0:00:00
     -------------------------------------- 145.3/145.3 kB 4.4 MB/s eta 0:00:00
     -------------------------------------- 269.6/269.6 kB 8.4 MB/s eta 0:00:00
     -------------------------------------- 168.9/168.9 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2021.11.1
    Uninstalling fsspec-2021.11.1:
      Successfully uninstalled fsspec-2021.11.1


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [6]:
def get_sentiment(text):
    #limit text to 512 characters
    text = text[:512]
    return pipe(text)[0]["label"]

In [7]:
df_reviews["sentiment"] = df_reviews.text.apply(get_sentiment)

In [8]:
df_reviews.to_csv("df_review_sentiment.csv")

In [14]:
from transformers import pipeline
#using model: https://huggingface.co/facebook/bart-large-mnli
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

candidate_labels = ['weather', 'service', 'food', 'price']


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceClassification: ['model.decoder.version', 'model.encoder.version']
- This IS expected if you are initializing TFBartForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForSequenceClassification for predictions without further training.


In [15]:
def get_class_probs(row):
    obj = classifier(row["text"][:512], candidate_labels, multi_label=True)
    row['weather_prob'] = obj["scores"][obj["labels"].index("weather")]
    row['service_prob'] = obj["scores"][obj["labels"].index("service")]
    row['food_prob'] = obj["scores"][obj["labels"].index("food")]
    row['price_prob'] = obj["scores"][obj["labels"].index("price")]
    
    return row



In [11]:
random_sample = df_reviews.groupby("sentiment").sample(n=200, random_state=42)

In [16]:
df_reviews_with_class_prob = random_sample.apply(get_class_probs, axis=1)

In [17]:
df_reviews_with_class_prob.to_csv("df_review_class_prob.csv")